## Google Services Account

These are some extra steps for service account.  Eventually want to get this working. 




In [ ]:
#This will import some required libraries.
import sys 
import ruamel.yaml
import os.path
import collections 
#This is your configuration file. 
kube_yaml='../../config/kubernetes.yaml'
with open(kube_yaml, 'r') as yaml:
    cf=ruamel.yaml.round_trip_load(yaml, preserve_quotes=True)
#This will allow us to import some useful utilities. 
if cf['general']['docker']:
    cf['general']['path']=cf['general']['docker_path']
else:
    cf['general']['path']=cf['general']['local_path']

sys.path.append(cf['general']['path']+"/kuberutils")   
import kuberutils as ku
#This just creates some additional varaibles. 
cf_k=ku.kubernetes_commands(cf)
cf_g=ku.gcloud_commands(cf)
#print(ruamel.yaml.dump(cf, sys.stdout, Dumper=ruamel.yaml.RoundTripDumper))

In [ ]:
#This will retreive common commands
cf_g=ku.gcloud_commands(cf)
#ruamel.yaml.dump(cf_g, sys.stdout, Dumper=ruamel.yaml.RoundTripDumper)
print(ruamel.yaml.dump(cf_g, sys.stdout, Dumper=ruamel.yaml.RoundTripDumper))

### Web Login

In order to use the web login, you have to open a terminal session.  Click on the `Jupyter` icon on the top left of the notebook and then click `new`->`terminal`.

Once in the terminal, issue the following command to login to your gmail account. It will provide you with a code you will then enter into the container. 

```
gcloud init
```


### Service Account Login
This is the easier way to manage login programatically.  Please be very careful about sharing this file and DON'T commit it!

Follow (this link)[https://cloud.google.com/compute/docs/access/create-enable-service-accounts-for-instances#createanewserviceaccount] to setup your service account. 

In you configuration file, set the name of your file here.  
`authorization_file: auth.json`

Then place the file in the `/config/gcloud`  directory.

https://cloud.google.com/iam/docs/granting-roles-to-service-accounts
gcloud iam service-accounts get-iam-policy \
kuberlytics@kuberlytics.iam.gserviceaccount.com \
--format json > policy.json

In [ ]:
ku.gcloud_command(cf_g['create_service_account'],cf)

In [ ]:
ku.gcloud_command(cf_g['create_key'],cf)

In [ ]:
!gcloud projects add-iam-policy-binding kuberlytics --member serviceAccount:kuberlytics@kuberlytics-2017.iam.gserviceaccount.com --role roles/editor

In [ ]:
gcloud compute instances set-service-account example-instance \
     --service-account my-sa-123@my-project-123.iam.gserviceaccount.com \
     --scopes compute-rw,storage-ro

In [ ]:
ku.gcloud_command(cf_g['get_policy'],cf)

In [ ]:
gcloud compute instances set-service-account example-instance \
     --service-account my-sa-123@my-project-123.iam.gserviceaccount.com \
     --scopes compute-rw,storage-ro

### Go to the services account page and then switch to: DwD: G Suite Domain-wide Delegation is enabled.

### Update the Policy to Bind to Service Account.  
https://cloud.google.com/sdk/gcloud/reference/beta/iam/service-accounts/add-iam-policy-binding 

So right now you have to manually update this. TBD have code that automates this update.  


```
{
  "bindings": [
    {
      "members": [
        "user:bob@gmail.com"
      ],
      "role": "roles/iam.serviceAccountActor"
    },
    {
      "members": [
        "user:bob@gmail.com"
      ],
      "role": "roles/owner"
    }
  ],
  "etag": "Bwdfdfdsssdf="
}
```

Note for future: Looks like this is in beta.  In future might be able to do via command line.  
```
gcloud beta iam service-accounts add-iam-policy-binding \
      my-iam-account@somedomain.com \
      --member='user:test-user@gmail.com' --role='roles/editor'

gcloud iam service-accounts add-iam-policy-binding

```

In [ ]:
#Run this once you have updated the policy.json
ku.gcloud_command(cf_g['set_policy'],cf)

In [ ]:
#Add a test to get the policy again and verify the binding. 

In [ ]:
#This just tests the service account login.
ku.gcloud_command(cf_g['login'],cf)